In [1]:
from dotenv import load_dotenv

load_dotenv(
    dotenv_path="/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env"
)
from api_helpers.config import Config
import pandas as pd

config = Config()

In [2]:
from api_helpers.clients import get_betfair_client, get_postgres_client

# bf_client = get_betfair_client()
pg_client = get_postgres_client()
import requests

2025-11-15T12:52:12Z | INFO - Logging configuration initialized with level: INFO


In [ ]:

TOKEN=''
BASE_URL='https://api.sportmonks.com/v3/football/'

In [4]:
requests.get(f'{BASE_URL}leagues?api_token={TOKEN}').json()

{'data': [{'id': 8,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Premier League',
   'active': True,
   'short_code': 'UK PL',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/8/8.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2025-11-09 16:30:00',
   'category': 1,
   'has_jerseys': False},
  {'id': 9,
   'sport_id': 1,
   'country_id': 462,
   'name': 'Championship',
   'active': True,
   'short_code': 'UK CHAMP',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/9/9.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2025-11-08 15:00:00',
   'category': 2,
   'has_jerseys': False},
  {'id': 12,
   'sport_id': 1,
   'country_id': 462,
   'name': 'League One',
   'active': True,
   'short_code': 'UK L1',
   'image_path': 'https://cdn.sportmonks.com/images/soccer/leagues/12/12.png',
   'type': 'league',
   'sub_type': 'domestic',
   'last_played_at': '2025-11-08 15:00:00',
   'category': 2,
 

In [24]:
LEAGUES ={
    'Premier League': 8,
    'Championship': 9,
    'League One': 12,
    'League Two': 14
}

In [24]:
import http.client

conn = http.client.HTTPSConnection("api.sportmonks.com")
payload = ''
headers = {}
conn.request("GET", f"/v3/football/seasons?api_token={TOKEN}", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

    {"data":[{"id":2,"sport_id":1,"league_id":8,"tie_breaker_rule_id":1526,"name":"2010\/2011","finished":true,"pending":false,"is_current":false,"starting_at":"2010-08-14","ending_at":"2011-05-22","standings_recalculated_at":"2023-05-24 08:28:07","games_in_current_week":false},{"id":3,"sport_id":1,"league_id":8,"tie_breaker_rule_id":1526,"name":"2013\/2014","finished":true,"pending":false,"is_current":false,"starting_at":"2013-08-17","ending_at":"2014-05-11","standings_recalculated_at":"2023-05-24 08:28:09","games_in_current_week":false},{"id":6,"sport_id":1,"league_id":8,"tie_breaker_rule_id":1526,"name":"2008\/2009","finished":true,"pending":false,"is_current":false,"starting_at":"2008-08-16","ending_at":"2009-05-24","standings_recalculated_at":"2023-11-30 13:38:48","games_in_current_week":false},{"id":7,"sport_id":1,"league_id":8,"tie_breaker_rule_id":1526,"name":"2012\/2013","finished":true,"pending":false,"is_current":false,"starting_at":"2012-08-18","ending_at":"2013-05-19","sta

In [33]:
def get_all_paginated_data(endpoint, token, process_item_func=None, includes=None):
    """
    Fetches all data from a paginated Sportmonks API endpoint.

    Args:
        endpoint (str): The API endpoint to fetch data from (e.g., 'seasons').
        token (str): Your Sportmonks API token.
        process_item_func (callable, optional): A function to process each item. Defaults to None.
        includes (list, optional): A list of relations to include in the response. Defaults to None.

    Returns:
        list: A list of all items retrieved from all pages.
    """
    all_items = []
    
    # Construct the initial URL
    # Check if the endpoint already has query parameters
    separator = '&' if '?' in endpoint else '?'
    url = f"{BASE_URL}{endpoint}{separator}api_token={token}"

    # Add includes if provided
    if includes:
        include_str = ",".join(includes)
        url += f"&include={include_str}"

    while url:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error fetching data from {url}. Status code: {response.status_code}")
            print(response.json())
            break

        data = response.json()

        if 'data' in data:
            for item in data['data']:
                if process_item_func:
                    processed_item = process_item_func(item)
                    all_items.append(processed_item)
                else:
                    all_items.append(item)
        
        if data.get('pagination', {}).get('has_more'):
            # The next_page URL from the API doesn't include the token or includes, so we add them
            url = data['pagination']['next_page'] + f'&api_token={token}'
            if includes:
                include_str = ",".join(includes)
                url += f"&include={include_str}"
        else:
            url = None
            
    return all_items

# Define a specific function to process a season item
def process_season(season):
    try:
        return {
            'id': season['id'],
            'league_id': season['league_id'],
            'name': season['name'],
            'start_year': int(season['name'].split('/')[0]),
            'end_year': int(season['name'].split('/')[1]),
        }
    except (ValueError, IndexError):
        # Handle cases where the name format is unexpected
        return {
            'id': season['id'],
            'league_id': season['league_id'],
            'name': season['name'],
            'start_year': None,
            'end_year': None,
        }

# --- Usage Example ---
# Call the generic function with the 'seasons' endpoint and the processing function
all_seasons_data = get_all_paginated_data('seasons', TOKEN, process_item_func=process_season)

# Convert the result to a pandas DataFrame
seasons_df = pd.DataFrame(all_seasons_data)

print(f"Total seasons found: {len(seasons_df)}")
print(seasons_df.head())
print(seasons_df.tail())

Total seasons found: 89
   id  league_id       name  start_year  end_year
0   2          8  2010/2011        2010      2011
1   3          8  2013/2014        2013      2014
2   6          8  2008/2009        2008      2009
3   7          8  2012/2013        2012      2013
4   8          8  2006/2007        2006      2007
       id  league_id       name  start_year  end_year
84  24263          8  2000/2001        2000      2001
85  25583          8  2025/2026        2025      2026
86  25648          9  2025/2026        2025      2026
87  25649         12  2025/2026        2025      2026
88  25650         14  2025/2026        2025      2026


In [9]:
seasons_df.sort_values(by=['league_id', 'start_year'])

,id,league_id,name,start_year,end_year
84,24263,8,2000/2001,2000,2001
83,24262,8,2001/2002,2001,2002
82,24261,8,2002/2003,2002,2003
81,24256,8,2003/2004,2003,2004
80,24255,8,2004/2005,2004,2005
...,...,...,...,...,...
67,18434,14,2021/2022,2021,2022
71,19795,14,2022/2023,2022,2023
75,21691,14,2023/2024,2023,2024
79,23673,14,2024/2025,2024,2025


In [11]:
import http.client

conn = http.client.HTTPSConnection("api.sportmonks.com")
payload = ''
headers = {}
conn.request("GET", f"/v3/football/fixtures?api_token={TOKEN}", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

    {"data":[{"id":463,"sport_id":1,"league_id":8,"season_id":2,"stage_id":2,"group_id":null,"aggregate_id":null,"round_id":43,"state_id":5,"venue_id":209,"name":"Tottenham Hotspur vs Manchester City","starting_at":"2010-08-14 11:45:00","result_info":"Game ended in draw.","leg":"1\/1","details":null,"length":90,"placeholder":false,"has_odds":false,"has_premium_odds":false,"starting_at_timestamp":1281786300},{"id":464,"sport_id":1,"league_id":8,"season_id":3,"stage_id":3,"group_id":null,"aggregate_id":null,"round_id":44,"state_id":5,"venue_id":230,"name":"Liverpool vs Stoke City","starting_at":"2013-08-17 11:45:00","result_info":"Liverpool won after full-time.","leg":"1\/1","details":null,"length":90,"placeholder":false,"has_odds":false,"has_premium_odds":false,"starting_at_timestamp":1376739900},{"id":465,"sport_id":1,"league_id":8,"season_id":2,"stage_id":2,"group_id":null,"aggregate_id":null,"round_id":43,"state_id":5,"venue_id":5,"name":"Aston Villa vs West Ham United","starting_at"

In [12]:
data

b'    {"data":[{"id":463,"sport_id":1,"league_id":8,"season_id":2,"stage_id":2,"group_id":null,"aggregate_id":null,"round_id":43,"state_id":5,"venue_id":209,"name":"Tottenham Hotspur vs Manchester City","starting_at":"2010-08-14 11:45:00","result_info":"Game ended in draw.","leg":"1\\/1","details":null,"length":90,"placeholder":false,"has_odds":false,"has_premium_odds":false,"starting_at_timestamp":1281786300},{"id":464,"sport_id":1,"league_id":8,"season_id":3,"stage_id":3,"group_id":null,"aggregate_id":null,"round_id":44,"state_id":5,"venue_id":230,"name":"Liverpool vs Stoke City","starting_at":"2013-08-17 11:45:00","result_info":"Liverpool won after full-time.","leg":"1\\/1","details":null,"length":90,"placeholder":false,"has_odds":false,"has_premium_odds":false,"starting_at_timestamp":1376739900},{"id":465,"sport_id":1,"league_id":8,"season_id":2,"stage_id":2,"group_id":null,"aggregate_id":null,"round_id":43,"state_id":5,"venue_id":5,"name":"Aston Villa vs West Ham United","starting

In [13]:
p = data.decode("utf-8")

In [15]:
print(p)

    {"data":[{"id":463,"sport_id":1,"league_id":8,"season_id":2,"stage_id":2,"group_id":null,"aggregate_id":null,"round_id":43,"state_id":5,"venue_id":209,"name":"Tottenham Hotspur vs Manchester City","starting_at":"2010-08-14 11:45:00","result_info":"Game ended in draw.","leg":"1\/1","details":null,"length":90,"placeholder":false,"has_odds":false,"has_premium_odds":false,"starting_at_timestamp":1281786300},{"id":464,"sport_id":1,"league_id":8,"season_id":3,"stage_id":3,"group_id":null,"aggregate_id":null,"round_id":44,"state_id":5,"venue_id":230,"name":"Liverpool vs Stoke City","starting_at":"2013-08-17 11:45:00","result_info":"Liverpool won after full-time.","leg":"1\/1","details":null,"length":90,"placeholder":false,"has_odds":false,"has_premium_odds":false,"starting_at_timestamp":1376739900},{"id":465,"sport_id":1,"league_id":8,"season_id":2,"stage_id":2,"group_id":null,"aggregate_id":null,"round_id":43,"state_id":5,"venue_id":5,"name":"Aston Villa vs West Ham United","starting_at"

In [20]:


conn.request("GET", f"/v3/football/fixtures/487?api_token={TOKEN}&include=statistics", payload, headers)
res = conn.getresponse()
data = res.read()
p = data.decode("utf-8")

In [27]:
start_date = "2023-01-01"
end_date = "2023-04-01"
conn.request("GET", f"/v3/football/fixtures/between/{start_date}/{end_date}?api_token={TOKEN}", payload, headers)
res = conn.getresponse()
data = res.read()
p = data.decode("utf-8")

In [34]:
start_date = "2023-01-01"
end_date = "2023-01-02" # Using a smaller date range for a quicker example

# Use your generic function to get all fixtures with participants
fixtures_endpoint = f"fixtures/between/{start_date}/{end_date}"
all_fixtures_with_participants = get_all_paginated_data(
    fixtures_endpoint, 
    TOKEN, 
    includes=['statistics']
)

# Convert the result to a pandas DataFrame
fixtures_with_participants_df = pd.DataFrame(all_fixtures_with_participants)

In [51]:
# The Core API has a different base URL
CORE_BASE_URL = 'https://api.sportmonks.com/v3/core/'

def get_all_paginated_core_data(endpoint, token):
    """
    Fetches all data from a paginated Sportmonks CORE API endpoint.
    """
    all_items = []
    url = f"{CORE_BASE_URL}{endpoint}?api_token={token}"

    while url:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error fetching data from {url}. Status code: {response.status_code}")
            print(response.json())
            break

        data = response.json()

        if 'data' in data:
            all_items.extend(data['data'])
        
        if data.get('pagination', {}).get('has_more'):
            url = data['pagination']['next_page'] + f'&api_token={token}'
        else:
            url = None
            
    return all_items

# --- Usage Example for Core Types ---
all_types = get_all_paginated_core_data('types', TOKEN)
types_df = pd.DataFrame(all_types)

print(f"Total types found: {len(types_df)}")
print(types_df.head())

Total types found: 1210
   id        name        code    developer_name model_type stat_group
0   1    1st Half    1st-half          1ST_HALF     period       None
1   2    2nd Half    2nd-half          2ND_HALF     period       None
2   3  Extra Time  extra-time                ET     period       None
3   5   Penalties   penalties  PENALTY_SHOOTOUT     period       None
4   6     Referee     referee           REFEREE    referee       None


In [52]:
types_df

,id,name,code,developer_name,model_type,stat_group
0,1,1st Half,1st-half,1ST_HALF,period,None
1,2,2nd Half,2nd-half,2ND_HALF,period,None
2,3,Extra Time,extra-time,ET,period,None
3,5,Penalties,penalties,PENALTY_SHOOTOUT,period,None
4,6,Referee,referee,REFEREE,referee,None
...,...,...,...,...,...,...
1205,87930,Match Fact Last 15 Goal Timings,last-15-goal-timings,MATCH_FACT_LAST_15_GOAL_TIMINGS,match_fact,
1206,87931,Match Fact Last 25 Goal Timings,last-25-goal-timings,MATCH_FACT_LAST_25_GOAL_TIMINGS,match_fact,
1207,87932,Match Fact Last 10 Cards Count in Match,last-10-cards-count-in-match,MATCH_FACT_LAST_10_CARDS_COUNT_IN_MATCH,match_fact,
1208,97352,Pressure Status,pressure-status,PRESSURE_STATUS,metadata,


In [53]:
types_df['model_type'].value_counts()

model_type
injury_suspension      562
statistic              192
match_fact             118
standing_rule           88
sub_event               41
prediction              36
metadata                30
position                25
standings               24
score                   14
event                   13
tie_breaker_rule         9
period                   8
standing_correction      7
lap                      7
transfer                 6
timeline                 5
lineup                   5
fee                      5
stage_type               4
referee                  4
highlight                3
stint                    2
pitstop                  2
Name: count, dtype: int64

In [55]:
statistics = types_df[types_df['model_type'] == 'statistic']
statistics

,id,name,code,developer_name,model_type,stat_group
32,34,Corners,corners,CORNERS,statistic,offensive
37,40,Captain,captain,CAPTAIN,statistic,overall
38,41,Shots Off Target,shots-off-target,SHOTS_OFF_TARGET,statistic,offensive
39,42,Shots Total,shots-total,SHOTS_TOTAL,statistic,offensive
40,43,Attacks,attacks,ATTACKS,statistic,offensive
...,...,...,...,...,...,...
1060,27274,Aerials,aerials,AERIALS,statistic,overall
1061,27275,Aerials Won Percentage,aerials-won-percentage,AERIALS_WON_PERCENTAGE,statistic,overall
1062,27276,Duels Won Percentage,duels-won-percentage,DUELS_WON_PERCENTAGE,statistic,offensive
1065,28036,Player Shot Conversion,player-shot-conversion,PLAYER_SHOT_CONVERSION,statistic,None


In [62]:
statistics[statistics['id'] == 78]

,id,name,code,developer_name,model_type,stat_group
74,78,Tackles,tackles,TACKLES,statistic,defensive


In [ ]:
18544648

In [38]:
fixtures_with_participants_df['statistics'].iloc[0]

[{'id': 7926234,
  'fixture_id': 18544648,
  'type_id': 86,
  'participant_id': 2,
  'data': {'value': 4},
  'location': 'home'},
 {'id': 7926226,
  'fixture_id': 18544648,
  'type_id': 64,
  'participant_id': 69,
  'data': {'value': 0},
  'location': 'away'},
 {'id': 7926206,
  'fixture_id': 18544648,
  'type_id': 60,
  'participant_id': 2,
  'data': {'value': 24},
  'location': 'home'},
 {'id': 7926227,
  'fixture_id': 18544648,
  'type_id': 78,
  'participant_id': 69,
  'data': {'value': 14},
  'location': 'away'},
 {'id': 7926233,
  'fixture_id': 18544648,
  'type_id': 83,
  'participant_id': 69,
  'data': {'value': 0},
  'location': 'away'},
 {'id': 7926218,
  'fixture_id': 18544648,
  'type_id': 84,
  'participant_id': 2,
  'data': {'value': 3},
  'location': 'home'},
 {'id': 7926224,
  'fixture_id': 18544648,
  'type_id': 45,
  'participant_id': 69,
  'data': {'value': 46},
  'location': 'away'},
 {'id': 7926207,
  'fixture_id': 18544648,
  'type_id': 41,
  'participant_id': 2,


In [58]:
import http.client

conn = http.client.HTTPSConnection("api.sportmonks.com")
payload = ''
headers = {}
conn.request("GET", f"/v3/football/fixtures/18544648?api_token={TOKEN}&include=statistics", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

    {"data":{"id":18544648,"sport_id":1,"league_id":9,"season_id":19793,"stage_id":77458014,"group_id":null,"aggregate_id":null,"round_id":275859,"state_id":5,"venue_id":339721,"name":"Blackburn Rovers vs Cardiff City","starting_at":"2023-01-01 12:00:00","result_info":"Blackburn Rovers won after full-time.","leg":"1\/1","details":null,"length":90,"placeholder":false,"has_odds":true,"has_premium_odds":false,"starting_at_timestamp":1672574400,"statistics":[{"id":7926234,"fixture_id":18544648,"type_id":86,"participant_id":2,"data":{"value":4},"location":"home"},{"id":7926226,"fixture_id":18544648,"type_id":64,"participant_id":69,"data":{"value":0},"location":"away"},{"id":7926206,"fixture_id":18544648,"type_id":60,"participant_id":2,"data":{"value":24},"location":"home"},{"id":7926227,"fixture_id":18544648,"type_id":78,"participant_id":69,"data":{"value":14},"location":"away"},{"id":7926233,"fixture_id":18544648,"type_id":83,"participant_id":69,"data":{"value":0},"location":"away"},{"id":

In [72]:
import http.client
import json

# The Fixture ID you are querying
FIXTURE_ID = "18544648"

# --- This is the key part ---
# Build a comprehensive 'include' string to get all related data.
# We use dot notation (e.g., 'statistics.type') to replace IDs
# with the actual data objects.

INCLUDES = [
    # --- For Statistics (what you have) ---
    "statistics.type",      # Replaces 'type_id' with the stat name (e.g., "Ball Possession")
    "statistics.participant", # Replaces 'participant_id' with the team object
    
    # --- For Players (what you asked for) ---
    "lineups.player",         # Gets the full player data for each player in the lineup
    "lineups.position",       # Gets the player's position name (e.g., "Defender")
    "lineups.type",           # Gets the lineup type (e.g., "starting_lineup", "bench")
    
    # --- For Other Game Details ---
    "events.type",            # Replaces 'type_id' in events with the event name (e.g., "Goal")
    "events.player",          # Gets the player who committed the event (e.g., goal scorer)
    "events.relatedPlayer",   # Gets the assist-provider (if any)
    "participants",           # Gets the full data for the two teams (home & away)
    # "participants.coach",     # Gets the coach for each team
    
    # --- For General Context ---
    "league",                 # Gets the full league data
    "season",                 # Gets the full season data
    # "venue",                  # Gets the full venue data
    # "referee"                 # Gets the full referee data
]

# Join the list into a single, comma-separated string
INCLUDE_STRING = ";".join(INCLUDES)

conn = http.client.HTTPSConnection("api.sportmonks.com")
payload = ''
headers = {}

# Construct the new, more powerful URL
URL = f"/v3/football/fixtures/{FIXTURE_ID}?api_token={TOKEN}&include={INCLUDE_STRING}"

print(URL)

print(f"Requesting URL: {URL}\n")

conn.request("GET", URL, payload, headers)
res = conn.getresponse()
data = res.read()
response_text = data.decode("utf-8")

# --- Bonus: Pretty-print the JSON to see the structure ---
try:
    parsed_json = json.loads(response_text)
    print(json.dumps(parsed_json, indent=2))
except json.JSONDecodeError:
    print("Could not parse JSON. Raw response:\n")
    print(response_text)

/v3/football/fixtures/18544648?api_token=4Yrn71jdChFheEPNW4jq5gd5B75WOitQRMBdAOu82djexQ4gP3ywrTSGv9R0&include=statistics.type;statistics.participant;lineups.player;lineups.position;lineups.type;events.type;events.player;events.relatedPlayer;participants;league;season
Requesting URL: /v3/football/fixtures/18544648?api_token=4Yrn71jdChFheEPNW4jq5gd5B75WOitQRMBdAOu82djexQ4gP3ywrTSGv9R0&include=statistics.type;statistics.participant;lineups.player;lineups.position;lineups.type;events.type;events.player;events.relatedPlayer;participants;league;season

{
  "data": {
    "id": 18544648,
    "sport_id": 1,
    "league_id": 9,
    "season_id": 19793,
    "stage_id": 77458014,
    "group_id": null,
    "aggregate_id": null,
    "round_id": 275859,
    "state_id": 5,
    "venue_id": 339721,
    "name": "Blackburn Rovers vs Cardiff City",
    "starting_at": "2023-01-01 12:00:00",
    "result_info": "Blackburn Rovers won after full-time.",
    "leg": "1/1",
    "details": null,
    "length": 90,
  

In [73]:
def parse_fixture_stats(json_string):
    """
    Parses the full fixture API response and organizes statistics
    into home and away dictionaries.
    """
    try:
        # Load the JSON string into a Python dictionary
        parsed_data = json.loads(json_string)
        
        # Access the main 'data' object
        fixture_data = parsed_data.get('data', {})
        
        # Access the 'statistics' list within the 'data' object
        statistics_list = fixture_data.get('statistics', [])
        
        if not statistics_list:
            print("No statistics found in the response.")
            return

        # Initialize empty dictionaries to hold the sorted stats
        home_stats = {}
        away_stats = {}
        
        # Get team names from the first stat objects we find for each
        # (This is a simpler way than using the 'participants' include)
        home_team_name = "Home Team"
        away_team_name = "Away Team"
        
        # Loop through every statistic object in the list
        for stat in statistics_list:
            try:
                # Get the common data points for this statistic
                stat_name = stat['type']['name']
                stat_value = stat['data']['value']
                location = stat['location']
                
                # Assign the stat to the correct dictionary
                if location == 'home':
                    home_stats[stat_name] = stat_value
                    # Try to get the team name just once
                    if home_team_name == "Home Team":
                        home_team_name = stat['participant']['name']
                elif location == 'away':
                    away_stats[stat_name] = stat_value
                    # Try to get the team name just once
                    if away_team_name == "Away Team":
                        away_team_name = stat['participant']['name']
                        
            except KeyError as e:
                print(f"Skipping a stat, missing expected key: {e}")
            except TypeError as e:
                print(f"Skipping a stat, data structure was not as expected: {e}")


        # --- All done, print the results ---
        
        print("--- Stats Parsing Complete ---")
        
        print(f"\nStats for {home_team_name} (Home):")
        # Use json.dumps for a clean "pretty-print"
        print(json.dumps(home_stats, indent=2))
        
        print(f"\nStats for {away_team_name} (Away):")
        print(json.dumps(away_stats, indent=2))

    except json.JSONDecodeError:
        print("Error: Could not decode the JSON string.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [75]:
p = parse_fixture_stats(response_text)

--- Stats Parsing Complete ---

Stats for Blackburn Rovers (Home):
{
  "Shots On Target": 4,
  "Throwins": 24,
  "Yellowcards": 3,
  "Shots Off Target": 4,
  "Assists": 1,
  "Redcards": 0,
  "Successful Passes Percentage": 81,
  "Hit Woodwork": 0,
  "Corners": 3,
  "Ball Possession %": 54,
  "Successful Interceptions": 22,
  "Tackles": 22,
  "Penalties": 0,
  "Successful Passes": 406,
  "Substitutions": 4,
  "Headers": 24,
  "Shots Total": 10,
  "Shots Outsidebox": 5,
  "Saves": 3,
  "Dangerous Attacks": 31,
  "Attacks": 78,
  "Challenges": 4,
  "Goals": 1,
  "Successful Headers": 15,
  "Counter Attacks": 0,
  "Shots Blocked": 2,
  "Fouls": 6,
  "Passes": 504,
  "Offsides": 2,
  "Treatments": 0,
  "Duels Won": 51,
  "Shots Insidebox": 5,
  "Interceptions": 5,
  "Total Crosses": 9,
  "Dribble Attempts": 11,
  "Key Passes": 9,
  "Accurate Crosses": 2,
  "Successful Dribbles": 4,
  "Successful Dribbles Percentage": 36
}

Stats for Cardiff City (Away):
{
  "Hit Woodwork": 0,
  "Tackles": 1

In [76]:
p